In [1]:
import pandas as pd
import numpy as np

# Corregir la ruta del archivo
df = pd.read_parquet("/home/donsson/proyectos/API/ventashistoricas52semanas.parquet") #movimiento  facturas
df_p = pd.read_parquet("/home/donsson/proyectos/API/costo_productos.parquet") #Costos unitarios
df_vp = pd.read_parquet("/home/donsson/proyectos/API/ventas_perdidas_2025.parquet") #ventas perdidas
vp_reales = pd.read_excel("/home/donsson/proyectos/INDICADOR NS/vp_agosto.xlsx") #vp reales


In [32]:
df.head()

,price_subtotal,quantity,product_name,invoice_name,date_invoice,store_name,store_id_num,Sucursal_tipo,origen,Bodega
0,622200.0,1.0,[DCE10935138] G935A10P FILTRO COMB. PARTICULA...,PV1E11138 Mostrador Cota T2/4321,2025-09-22 12:16:39,PRINCIPAL COTA,1,MOSTRADOR,<NA>,PRINCIPAL COTA
1,68400.0,2.0,[DAC00191020] DAC191KIT FILTRO CABINA - FOTON ...,FMDE3939,2025-09-22 12:14:12,SUCURSAL MEDELLIN,8,VENDEDOR EXTERNO,BD5,SUCURSAL MEDELLIN
2,36900.0,3.0,[DAE01057189] GA1057 FILTRO AIRE CHEVROLET N20...,FMDE3938,2025-09-22 12:12:13,SUCURSAL MEDELLIN,8,VENDEDOR EXTERNO,BD5,SUCURSAL MEDELLIN
3,219000.0,5.0,[DAB08002025] DA8002 FILTRO AIRE FOTON OLIN 4....,FMDE3938,2025-09-22 12:12:13,SUCURSAL MEDELLIN,8,VENDEDOR EXTERNO,BD5,SUCURSAL MEDELLIN
4,840000.0,50.0,[DAE01142189] GA1142 FILTRO AIRE - FOTON (189 ...,FMDE3938,2025-09-22 12:12:13,SUCURSAL MEDELLIN,8,VENDEDOR EXTERNO,BD5,SUCURSAL MEDELLIN


# EDA

## Facturas

In [2]:
import re
import unicodedata



mapa_sucursales = {
    'BD1': 'PRINCIPAL COTA',
    'BD2': 'SUCURSAL CALLE 6',
    'BD3': 'SUCURSAL VALLADOLID',
    'BD4': 'SUCURSAL NORTE',
    'BD5': 'SUCURSAL MEDELLIN',
    'BD6': 'SUCURSAL BARRANQUILLA',
    'BD7':' SUCURSAL BUCARAGAMNGA ',
    'BD8':'MOSTRADOR COTA',
    'BD9':'SUCURSAL CALI',
    'BD11': 'CUMMINS DE LOS ANDES',
    'BD12': 'CUMMINS DE LOS ANDES MEDELLIN',
    'BD13': 'CSS CONSTRUCTORES',
    'BD14': 'ANTIOQUEÑA DE LUBRICANTES SGP SAS',
    'BD15': 'ASEO CAPITAL',
    'BD16': 'COVIANDES',
    'BD18': 'CEMEX BOSA',
    'BD19': 'Cemex ibague (eliminar)',
    'BD21': 'CEMEX CENTENARIO',
    'BD22': 'PENDIENTE POR FACTURAR CIERRE CONT.',
    'BD24': 'FILTRO EN REPROCESO',
    'BD26': 'PRESTAMOS INTERNOS',
    'BD31': 'EXP.CTP',
    'BD32': 'EXP. ECUADOR',
    'BD34':'EXP.DONSSON.USA',
    'BD40': 'DESCUADRE B1',
    'BD MP':'MATERIA PRIMA',
    'BD41':'BODEGA VENDEDOR WILMER GIL',
    'BD33':'EXP.REP.DOMINICANA',
    'BD42':'STOCK SIGMA ENERGY'
    # Agrega más códigos si los encuentras en tus datos, siguiendo este formato.
    }




def clasificar_sucursal(invoice_name):
    if pd.isna(invoice_name):
        return "VENDEDOR EXTERNO"
    if "Mostrador" in invoice_name:
        return "MOSTRADOR"
    return "VENDEDOR EXTERNO"
    
df["Sucursal_tipo"] = df["invoice_name"].apply(clasificar_sucursal)




#Quitar las facturas que salen como SO son no vendibles
df = df[~(df["origin"].str.contains("SO"))] 


# Normalizar los separadores para que todos sean iguales
df["origin_norm"] = df["origin"].str.replace(r"[\\/]", "/", regex=True)

# Extraer solo el BD y el número (ejemplo: BD11, BD9, etc.)
df["origen"] = df["origin_norm"].str.extract(r"^(BD\d+)")

df["Bodega"] = df["origen"].map(mapa_sucursales)
df["Bodega"] = df["Bodega"].fillna(df["store_name"])

df = df.drop(columns=["origin_norm"])


df = df.drop(columns=["origin"])

df_externos_bogota = df[(df["store_name"]=="PRINCIPAL COTA") & (df["Sucursal_tipo"]=="VENDEDOR EXTERNO")]

df = df.drop(df_externos_bogota.index)


In [3]:
df.groupby(["store_name","Bodega","Sucursal_tipo"])["quantity"].sum()

store_name             Bodega                             Sucursal_tipo   
PRINCIPAL COTA         PRINCIPAL COTA                     MOSTRADOR           15976.0
SUCURSAL BARRANQUILLA  BODEGA VENDEDOR WILMER GIL         VENDEDOR EXTERNO      113.0
                       SUCURSAL BARRANQUILLA              MOSTRADOR           23726.0
                                                          VENDEDOR EXTERNO    74389.0
                       SUCURSAL CALI                      VENDEDOR EXTERNO        2.0
SUCURSAL BUCARAMANGA    SUCURSAL BUCARAGAMNGA             VENDEDOR EXTERNO    46281.0
                       PRINCIPAL COTA                     VENDEDOR EXTERNO        2.0
                       SUCURSAL BUCARAMANGA               MOSTRADOR           18204.0
SUCURSAL CALI          BODEGA VENDEDOR WILMER GIL         VENDEDOR EXTERNO     1928.0
                       EXP.REP.DOMINICANA                 VENDEDOR EXTERNO     1961.0
                       PRINCIPAL COTA                     VENDEDO

In [39]:
df[df["product_name"].str.contains("DCE00833189")].sort_values(by=["quantity"],ascending=False)

,price_subtotal,quantity,product_name,invoice_name,date_invoice,store_name,store_id_num,Sucursal_tipo,origen,Bodega
95903,715000.0,50.0,[DCE00833189] G833 FILTRO COMBUSTIBLE CHEVROLE...,FBAQ13002,2025-04-02 21:04:23,SUCURSAL BARRANQUILLA,10,VENDEDOR EXTERNO,BD6,SUCURSAL BARRANQUILLA
6350,715000.0,50.0,[DCE00833189] G833 FILTRO COMBUSTIBLE CHEVROLE...,PV2E43208 Mostrador Calle 6 T2/21164,2025-09-10 17:49:58,SUCURSAL CALLE 6,2,MOSTRADOR,<NA>,SUCURSAL CALLE 6
186890,514800.0,36.0,[DCE00833189] G833 FILTRO COMBUSTIBLE CHEVROLE...,PV1E8545 Mostrador Cota T2/1936,2024-10-15 20:33:42,PRINCIPAL COTA,1,MOSTRADOR,<NA>,PRINCIPAL COTA
114669,540000.0,30.0,[DCE00833189] G833 FILTRO COMBUSTIBLE CHEVROLE...,PV2E37805 Mostrador Calle 6 T3/21299,2025-02-28 14:32:09,SUCURSAL CALLE 6,2,MOSTRADOR,<NA>,SUCURSAL CALLE 6
52387,477000.0,30.0,[DCE00833189] G833 FILTRO COMBUSTIBLE CHEVROLE...,PV2E40849 Mostrador Calle 6 T3/22423,2025-06-20 14:02:51,SUCURSAL CALLE 6,2,MOSTRADOR,<NA>,SUCURSAL CALLE 6
...,...,...,...,...,...,...,...,...,...,...
196105,18000.0,1.0,[DCE00833189] G833 FILTRO COMBUSTIBLE CHEVROLE...,PV2E33524 Mostrador Calle 6 T1/21371,2024-09-26 15:30:50,SUCURSAL CALLE 6,2,MOSTRADOR,<NA>,SUCURSAL CALLE 6
195792,18000.0,1.0,[DCE00833189] G833 FILTRO COMBUSTIBLE CHEVROLE...,PV6E14205 Mostrador Barranquilla T2/20759,2024-09-26 19:47:27,SUCURSAL BARRANQUILLA,10,MOSTRADOR,<NA>,SUCURSAL BARRANQUILLA
195668,18000.0,1.0,[DCE00833189] G833 FILTRO COMBUSTIBLE CHEVROLE...,PV6E14208 Mostrador Barranquilla T2/20763,2024-09-26 21:16:52,SUCURSAL BARRANQUILLA,10,MOSTRADOR,<NA>,SUCURSAL BARRANQUILLA
195195,18000.0,1.0,[DCE00833189] G833 FILTRO COMBUSTIBLE CHEVROLE...,PV6E14220 Mostrador Barranquilla T2/20775,2024-09-27 17:52:12,SUCURSAL BARRANQUILLA,10,MOSTRADOR,<NA>,SUCURSAL BARRANQUILLA


In [29]:
df_externos_bogota = df[(df["store_name"]=="SUCURSAL CALI") & (df["Bodega"]=="EXP.REP.DOMINICANA")]

df_externos_bogota

,price_subtotal,quantity,product_name,invoice_name,date_invoice,store_name,store_id_num,Sucursal_tipo,origen,Bodega
55231,101600.0,2.0,[DAR03349025] DA3349 FILTRO AIRE 1_ COMPRESOR ...,FCAL4648,2025-06-16 14:09:12,SUCURSAL CALI,15,VENDEDOR EXTERNO,BD33,EXP.REP.DOMINICANA
55232,331200.0,12.0,[DAB04028025] DA4028 FILTRO AIRE 2_ KOMATSU (0...,FCAL4648,2025-06-16 14:09:12,SUCURSAL CALI,15,VENDEDOR EXTERNO,BD33,EXP.REP.DOMINICANA
55233,48300.0,1.0,[DAE09022025] DA9022 FILTRO AIRE DONSSON - LOV...,FCAL4648,2025-06-16 14:09:12,SUCURSAL CALI,15,VENDEDOR EXTERNO,BD33,EXP.REP.DOMINICANA
55234,139800.0,3.0,[DAB08114025] DA8114 FILTRO AIRE PRIMARIO EXCA...,FCAL4648,2025-06-16 14:09:12,SUCURSAL CALI,15,VENDEDOR EXTERNO,BD33,EXP.REP.DOMINICANA
55235,288000.0,2.0,[DAB08069025] DA8069 FILTRO AIRE CAMIONES RENA...,FCAL4648,2025-06-16 14:09:12,SUCURSAL CALI,15,VENDEDOR EXTERNO,BD33,EXP.REP.DOMINICANA
...,...,...,...,...,...,...,...,...,...,...
55303,3800000.0,125.0,"[DAB04690025] DA4690 FILTRO AIRE 2o J.DEERE, F...",FCAL4648,2025-06-16 14:09:12,SUCURSAL CALI,15,VENDEDOR EXTERNO,BD33,EXP.REP.DOMINICANA
55304,1153200.0,24.0,[DAB02028025] DA2028 FILTRO AIRE 1_ KOMATSU (0...,FCAL4648,2025-06-16 14:09:12,SUCURSAL CALI,15,VENDEDOR EXTERNO,BD33,EXP.REP.DOMINICANA
55305,2062500.0,125.0,[DAB14772025] DA4772A FILTRO AIRE INTERNO BOBC...,FCAL4648,2025-06-16 14:09:12,SUCURSAL CALI,15,VENDEDOR EXTERNO,BD33,EXP.REP.DOMINICANA
55306,2237500.0,125.0,[DAB02738025] DA2738 FILTRO AIRE CORTACESPED J...,FCAL4648,2025-06-16 14:09:12,SUCURSAL CALI,15,VENDEDOR EXTERNO,BD33,EXP.REP.DOMINICANA


## Ventas perdidas

In [5]:
import pandas as pd
import numpy as np

# ===============================
# Filtrar almacenamiento agotado
# ===============================
df_vp = df_vp[df_vp["almacenamiento_tipo"].str.lower() == "agotado"]

# ===============================
# Asegurar tipos correctos
# ===============================
df_vp = df_vp.copy()
df_vp["fecha"] = pd.to_datetime(df_vp["fecha"], errors="coerce")

# Numéricos
for col in ["cantidad", "cantidad_existencia", "cantidad_reservada"]:
    df_vp[col] = pd.to_numeric(df_vp[col], errors="coerce").fillna(0).clip(lower=0)

# ===============================
# Reglas Odoo vectorizadas
# ===============================
is_cot = df_vp["origen"].fillna("").str.lower() == "cotizacion"
ignore_mask = df_vp["cantidad"] >= 100

ajuste = np.where(
    is_cot,
    df_vp["cantidad"] - df_vp["cantidad_existencia"] - df_vp["cantidad_reservada"],
    df_vp["cantidad"] - df_vp["cantidad_reservada"]
)

# Aplicar reglas de descarte y piso en cero
ajuste = np.where(ignore_mask, 0, ajuste)
ajuste = np.where(ajuste > 0, ajuste, 0)

df_vp["ventas_perdidas"] = ajuste.astype(float)

# ===============================
# Columnas temporales
# ===============================
df_vp["Semana"] = df_vp["fecha"].dt.to_period("W").dt.start_time
df_vp["ano"]   = df_vp["Semana"].dt.year
df_vp["mes"]   = df_vp["Semana"].dt.month
df_vp["dia"]   = df_vp["Semana"].dt.day

# ===============================
# Filtro adicional: excluir SERV y CARCASA
# ===============================
mask_excluir = ~df_vp["product_ref"].str.contains("SERV|CARCASA", case=False, na=False)
df_vp = df_vp[mask_excluir]

# ===============================
# Agrupación por tienda + producto + semana
# ===============================
lost_by_week = (
    df_vp.groupby(["store_name", "product_ref", "Semana", "ano", "mes", "dia"])
    .agg(
        lost_sales=("ventas_perdidas", "sum"),   # suma total de ventas perdidas
        veces_vp=("ventas_perdidas", "count")    # número de veces que hubo pérdida
    )
    .reset_index()
)

# Mostrar resultado agrupado
vp_week = lost_by_week


vp_week["product_ref"].nunique()

4229

In [6]:
vp_reales["product_ref"] = vp_reales["Descripcion"].str.extract(r"\[([A-Z0-9]+)\]")
vp_reales.head()

# Asegurar que ambos son strings para evitar problemas
vp_week["product_ref"] = vp_week["product_ref"].astype(str)
vp_reales["product_ref"] = vp_reales["product_ref"].astype(str)

# 1. Obtener listas únicas
refs_week = set(vp_week["product_ref"].unique())
refs_real = set(vp_reales["product_ref"].unique())

# 2. Diferencia: los que están en vp_week pero no en vp_real
refs_extra = refs_week - refs_real

# 3. Filtrar el dataframe para verlos completos
df_discrepantes = vp_week[vp_week["product_ref"].isin(refs_extra)]


df_discrepantes = df_discrepantes[(df_discrepantes["mes"]==8) & (df_discrepantes["lost_sales"]>0) ]
df_discrepantes = df_discrepantes.groupby("product_ref").agg({"lost_sales":"sum"})
print("Cantidad de vp que no deberia tomar:", df_discrepantes["lost_sales"].sum())
df_discrepantes #Los productos que no se movieron hace mucho tiempo no salen en el analisis de ns

Cantidad de vp que no deberia tomar: 47.0


,lost_sales
product_ref,
DAB28118025,27.0
DAR12123UHE,2.0
DCS00342118,2.0
DCS00342186,1.0
DLS00105011,1.0
DLX00393020,14.0


In [7]:
mes = 9
vp_agosot_2025 = vp_week[(vp_week["ano"]==2025)& (vp_week["mes"]==mes)]
vp_agosot_2025.to_excel(f"/home/donsson/proyectos/MODELO ABASTECIMIENTO/exceles/vp_definitivasparaanalisis{mes}.xlsx")

vp_agosot_2025.groupby("store_name")["lost_sales"].sum()

store_name
PRINCIPAL COTA           1316.0
SUCURSAL BARRANQUILLA     833.0
SUCURSAL BUCARAMANGA      535.0
SUCURSAL CALI             795.0
SUCURSAL CALLE 6          776.0
SUCURSAL MEDELLIN         586.0
SUCURSAL NORTE            411.0
SUCURSAL VALLADOLID       350.0
Name: lost_sales, dtype: float64

# UNION

## EMA SEMANAL CON VP SEMANALES (SOLO 2025)

In [8]:
# ===============================
# Procesar ventas normales
# ===============================
df_sales = df.copy()
df_sales["date_invoice"] = pd.to_datetime(df_sales["date_invoice"], errors="coerce")


# Referncia de producto
df_sales["product_ref"] = df_sales["product_name"].str.extract(r"\[([A-Z0-9]+)\]")


# Columnas temporales igual que en df_vp
df_sales["Semana"] = df_sales["date_invoice"].dt.to_period("W").dt.start_time
df_sales["ano"]    = df_sales["Semana"].dt.year
df_sales["mes"]    = df_sales["Semana"].dt.month
df_sales["dia"]    = df_sales["Semana"].dt.day

# ===============================
# Agrupación por tienda + producto + semana
# ===============================
sales_by_week = (
    df_sales.groupby(["store_name", "product_ref", "Semana", "ano", "mes", "dia"], as_index=False)
    .agg(
        sales=("quantity", "sum"),              # suma de unidades vendidas
        num_facturas=("date_invoice", "nunique")  # cantidad de facturas distintas
    )
)


# Resultado
sales_by_week.sample(10)



,store_name,product_ref,Semana,ano,mes,dia,sales,num_facturas
29385,SUCURSAL BUCARAMANGA,BLS00213125,2025-07-28,2025,7,28,1.0,1
24129,SUCURSAL BARRANQUILLA,DCS30536137,2025-03-24,2025,3,24,1.0,1
89153,SUCURSAL VALLADOLID,BCS00014125,2025-01-06,2025,1,6,1.0,1
10863,SUCURSAL BARRANQUILLA,BHE00806125,2025-05-12,2025,5,12,1.0,1
91591,SUCURSAL VALLADOLID,BLS00038125,2025-05-05,2025,5,5,1.0,1
66052,SUCURSAL MEDELLIN,BCS00197125,2024-12-09,2024,12,9,1.0,1
44149,SUCURSAL CALI,DAB09056025,2024-11-25,2024,11,25,2.0,1
43354,SUCURSAL CALI,DAB04919025,2025-07-14,2025,7,14,1.0,1
44763,SUCURSAL CALI,DAE04270025,2025-02-24,2025,2,24,1.0,1
49071,SUCURSAL CALLE 6,BCS00161125,2025-04-07,2025,4,7,4.0,3


In [9]:
df_merged = pd.merge(
    sales_by_week[["store_name", "product_ref", "Semana", "sales","num_facturas"]],
    vp_week[["store_name", "product_ref", "Semana", "lost_sales","veces_vp"]],
    on=["store_name", "product_ref", "Semana"],
    how="outer"
).fillna(0)


In [10]:
df_merged["año"]    = df_merged["Semana"].dt.year
df_merged["mes"]    = df_merged["Semana"].dt.month
df_merged["dia"]    = df_merged["Semana"].dt.day


df_merged.head(10)

,store_name,product_ref,Semana,sales,num_facturas,lost_sales,veces_vp,año,mes,dia
0,PRINCIPAL COTA,AGB0GRAS030,2025-07-28,0.0,0.0,1.0,1.0,2025,7,28
1,PRINCIPAL COTA,AHB0TO30132,2025-01-13,0.0,0.0,1.0,1.0,2025,1,13
2,PRINCIPAL COTA,AHB80W90030,2025-07-07,0.0,0.0,1.0,1.0,2025,7,7
3,PRINCIPAL COTA,AHB80W90050,2025-08-11,0.0,0.0,1.0,1.0,2025,8,11
4,PRINCIPAL COTA,AHBNTO68030,2024-10-21,1.0,1.0,0.0,0.0,2024,10,21
5,PRINCIPAL COTA,AHBNTO68030,2025-01-27,1.0,1.0,1.0,1.0,2025,1,27
6,PRINCIPAL COTA,AHBNTO68030,2025-02-03,0.0,0.0,1.0,1.0,2025,2,3
7,PRINCIPAL COTA,AHBNTO68030,2025-02-10,0.0,0.0,1.0,1.0,2025,2,10
8,PRINCIPAL COTA,AHBNTO68030,2025-02-17,0.0,0.0,4.0,1.0,2025,2,17
9,PRINCIPAL COTA,AHBNTO68030,2025-03-24,0.0,0.0,1.0,1.0,2025,3,24


## NORMALIZAR DF DE COSTOS

In [11]:
df_p["product_ref"] = df_p["product_name"].str.extract(r"\[([A-Z0-9]+)\]")

df_p_unique = (
    df_p[["product_ref", "producto_costo_unitario"]]
    .drop_duplicates(subset=["product_ref"])
)



df_p["product_ref"].nunique()

4835

### UNIR COSTO

In [12]:
df_merge_def = pd.merge(
    df_merged,
    df_p_unique,
    on="product_ref",
    how="left"
).fillna(0)


merge_def = df_merge_def[df_merge_def["producto_costo_unitario"] !=0].copy() #Eliminar productos sin costos unitarios



In [13]:
merge_def.sample(10)

,store_name,product_ref,Semana,sales,num_facturas,lost_sales,veces_vp,año,mes,dia,producto_costo_unitario
64810,SUCURSAL CALI,DAB09098025,2025-09-08,0.0,0.0,1.0,1.0,2025,9,8,32393.89
7647,PRINCIPAL COTA,DAB08032115,2025-05-12,0.0,0.0,1.0,1.0,2025,5,12,6807.50
69044,SUCURSAL CALLE 6,ALQ15W40030,2025-01-06,2.0,1.0,0.0,0.0,2025,1,6,26339.88
126758,SUCURSAL VALLADOLID,BLS00498125,2025-07-28,2.0,1.0,0.0,0.0,2025,7,28,18432.37
52565,SUCURSAL BUCARAMANGA,DAP04020025,2025-03-17,0.0,0.0,2.0,2.0,2025,3,17,7625.26
3409,PRINCIPAL COTA,BCS10352125,2025-01-20,0.0,0.0,1.0,1.0,2025,1,20,43472.40
25583,SUCURSAL BARRANQUILLA,BLS00670125,2025-02-17,0.0,0.0,4.0,4.0,2025,2,17,20123.29
5746,PRINCIPAL COTA,BLS00514125,2025-08-11,1.0,1.0,0.0,0.0,2025,8,11,36280.33
130867,SUCURSAL VALLADOLID,DAR02265025,2025-06-30,2.0,2.0,0.0,0.0,2025,6,30,42286.93
24731,SUCURSAL BARRANQUILLA,BLS00294125,2025-02-03,2.0,2.0,0.0,0.0,2025,2,3,21082.18


### PRUEBA EMA 1

In [ ]:
import pandas as pd
import numpy as np

def compute_demand_and_ema(df,
                           alpha=0.20,        # peso de EMA
                           n_init_weeks=12,   # semanas para inicializar EMA
                           week_col="Semana",
                           sales_col="sales",
                           lost_col="lost_sales"):
    df = df.copy()

    # ---------- Asegurar tipos y semana iniciando lunes ----------
    df[week_col] = pd.to_datetime(df[week_col], errors="coerce")
    df[week_col] = df[week_col].dt.to_period('W-MON').dt.start_time

    # Asegurar numéricos
    df[sales_col] = pd.to_numeric(df[sales_col], errors="coerce").fillna(0)
    df[lost_col]  = pd.to_numeric(df[lost_col], errors="coerce").fillna(0)

    # ---------- Crear combinaciones completas ----------
    stores   = df["store_name"].unique()
    products = df["product_ref"].unique()
    weeks    = df[week_col].unique()

    full_index = pd.MultiIndex.from_product([stores, products, weeks],
                                            names=["store_name", "product_ref", week_col])

    df = df.set_index(["store_name", "product_ref", week_col]).reindex(full_index).reset_index()

    # Rellenar ventas y perdidas con 0 en combinaciones faltantes
    df[sales_col] = df[sales_col].fillna(0)
    df[lost_col]  = df[lost_col].fillna(0)

    # Ordenar
    df = df.sort_values(["store_name", "product_ref", week_col])

    out_groups = []

    # ---------- Calcular demanda ajustada y EMA ----------
    for (store, prod), g in df.groupby(["store_name", "product_ref"], sort=False):
        g = g.sort_values(week_col).reset_index(drop=True)
        sales = g[sales_col].to_numpy(dtype=float)
        lost  = g[lost_col].to_numpy(dtype=float)

        L = len(g)
        demanda = np.zeros(L, dtype=float)
        ema_arr = np.zeros(L, dtype=float)

        if L == 0:
            out_groups.append(g)
            continue

        # Inicialización EMA: promedio de primeras n_init_weeks ventas
        init_n = min(n_init_weeks, L)
        ema_prev = float(np.nanmean(sales[:init_n])) if init_n > 0 else 0.0
        if np.isnan(ema_prev):
            ema_prev = 0.0

        for i in range(L):
            s = sales[i]
            l = lost[i]

            # Regla 1
            if s >= 2.0 * l:
                demand_candidate = s + l
                demand = min(demand_candidate, 1.5 * s) if s > 0 else demand_candidate
            else:
                # Regla 2
                demand = s + 0.5 * ema_prev

            # Guardar redondeando
            demanda[i] = round(demand, 6)

            # Calcular EMA
            ema = alpha * demand + (1.0 - alpha) * ema_prev
            ema = round(ema, 6)
            ema_arr[i] = ema

            # actualizar para siguiente
            ema_prev = ema

        # Añadir columnas al grupo
        g = g.copy()
        g["demanda_ajustada"] = demanda
        g["EMA"] = ema_arr

        out_groups.append(g)

        

    # Concat resultados
    result = pd.concat(out_groups, ignore_index=True, sort=False)
    result = result.sort_values(["store_name", "product_ref", week_col]).reset_index(drop=True)
    return result

# ------------------ USO ------------------
df_with_demand1 = compute_demand_and_ema(merge_def, alpha=0.2, n_init_weeks=16)

#APROX2min

In [15]:
def add_deviation_and_classification(df, ema_col="EMA", sales_col="sales", week_col="Semana", n_weeks=12):
    df = df.copy()
    out_groups = []

    for (store, prod), g in df.groupby(["store_name", "product_ref"], sort=False):
        g = g.sort_values(week_col).reset_index(drop=True)
        
        sales = g[sales_col].to_numpy(dtype=float)
        ema   = g[ema_col].to_numpy(dtype=float)
        L = len(sales)

        # Inicializamos columnas
        g["desviacion_ema%"] = 0.0
        g["coef_ventas"] = 0.0

        if L < 2:
            out_groups.append(g)
            continue

        for i in range(L):
            # Usar solo hasta la semana actual (i)
            window_start = max(0, i - n_weeks)       # ventana de 12 semanas
            window_sales = sales[window_start:i]     # semanas previas
            window_ema   = ema[window_start:i]

            if len(window_sales) < 2:
                continue

            promedio_prev = np.mean(window_sales)
            if promedio_prev == 0:
                promedio_prev = 1e-6

            # --- Desviación usando EMAs previos ---
            desv = np.std(window_ema, ddof=1) / promedio_prev if promedio_prev > 0 else 0

            # --- Coeficiente de ventas ---
            coef = (max(window_sales) - min(window_sales)) / promedio_prev if promedio_prev > 0 else 0

            g.at[i, "desviacion_ema%"] = desv
            g.at[i, "coef_ventas"] = coef

        out_groups.append(g)

    return pd.concat(out_groups, ignore_index=True)





df_with_demand1 = add_deviation_and_classification(df_with_demand1)
df_with_demand1[["store_name","product_ref","Semana","num_facturas","sales","lost_sales","demanda_ajustada","EMA","desviacion_ema%","coef_ventas"]].sample(10)

# Demora mas o menos 6 min

,store_name,product_ref,Semana,num_facturas,sales,lost_sales,demanda_ajustada,EMA,desviacion_ema%,coef_ventas
1703352,SUCURSAL VALLADOLID,DAR02695025,2025-06-10,1.0,5.0,0.0,5.0,1.414084,0.552067,8.571429
791929,SUCURSAL CALI,DAE01130122,2024-10-08,NaN,0.0,0.0,0.0,0.000000,0.000000,0.000000
420163,SUCURSAL BARRANQUILLA,DCS10270137,2025-04-29,NaN,0.0,0.0,0.0,0.222564,0.305838,4.000000
903713,SUCURSAL CALLE 6,BAE08124125,2024-11-26,NaN,0.0,0.0,0.0,0.000000,0.000000,0.000000
551319,SUCURSAL BUCARAMANGA,DAB09166025,2024-12-17,NaN,0.0,0.0,0.0,0.239484,0.393115,7.200000
1188829,SUCURSAL MEDELLIN,BLS00670125,2025-06-17,1.0,1.0,0.0,1.0,1.108077,0.266645,2.823529
1215076,SUCURSAL MEDELLIN,DAC00054189,2025-09-09,NaN,0.0,0.0,0.0,0.000000,0.000000,0.000000
447568,SUCURSAL BUCARAMANGA,BAB08157125,2025-05-27,NaN,0.0,0.0,0.0,0.067109,0.811194,12.000000
13452,PRINCIPAL COTA,BAC00146125,2025-07-15,NaN,0.0,0.0,0.0,0.000000,0.000000,0.000000
60148,PRINCIPAL COTA,BHE00204125,2025-08-05,NaN,0.0,0.0,0.0,0.000000,0.000000,0.000000


In [16]:
print(f"productos_unicos",df_with_demand1["product_ref"].nunique())

productos_unicos 4137


In [17]:
df_with_demand1["semana_num"] = df_with_demand1["Semana"].dt.isocalendar().week
#df_with_demand1= df_with_demand1[df_with_demand1["EMA"]>1]

In [18]:
df_demand_2025 = df_with_demand1[df_with_demand1["año"]==2025] #Ajustar año


demand_2025 = df_demand_2025[["store_name","product_ref","año","semana_num","num_facturas","EMA","demanda_ajustada","producto_costo_unitario","desviacion_ema%","coef_ventas"]] #"producto_costo_unitario","demanda_ajustada"

demand_2025_37 = demand_2025[demand_2025["semana_num"]==37]  #Ajustar semana numero


demand_2025_37.to_csv("ema_mio202537.csv")


filtro_bq1 = demand_2025_37[demand_2025["store_name"]=="SUCURSAL BARRANQUILLA"].sort_values(by=("EMA"), ascending=False)

filtro_bq1.head(30)

/tmp/ipykernel_47175/4053637058.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtro_bq1 = demand_2025_37[demand_2025["store_name"]=="SUCURSAL BARRANQUILLA"].sort_values(by=("EMA"), ascending=False)


,store_name,product_ref,año,semana_num,num_facturas,EMA,demanda_ajustada,producto_costo_unitario,desviacion_ema%,coef_ventas
317786,SUCURSAL BARRANQUILLA,DAB02570025,2025.0,37,7.0,38.554926,10.0,13507.71,0.193115,3.935484
301674,SUCURSAL BARRANQUILLA,BLS00037125,2025.0,37,9.0,33.981446,22.0,32797.97,0.112285,1.585421
334693,SUCURSAL BARRANQUILLA,DAB14570025,2025.0,37,6.0,30.869047,22.0,9774.98,0.626906,5.621262
318952,SUCURSAL BARRANQUILLA,DAB02772025,2025.0,37,4.0,30.471810,7.0,10680.97,0.170460,3.096774
318263,SUCURSAL BARRANQUILLA,DAB02666025,2025.0,37,6.0,25.581851,26.0,11371.65,0.107747,2.157303
261182,SUCURSAL BARRANQUILLA,BCS00035125,2025.0,37,11.0,23.910288,15.0,20638.72,0.158093,2.307692
273160,SUCURSAL BARRANQUILLA,BCS10035125,2025.0,37,7.0,22.682642,11.0,20956.60,0.135993,2.476190
260758,SUCURSAL BARRANQUILLA,BCS00025125,2025.0,37,5.0,19.326151,6.0,31058.67,0.081797,1.483636
263567,SUCURSAL BARRANQUILLA,BCS00249125,2025.0,37,8.0,19.151567,12.0,27520.25,0.112979,1.811321
321390,SUCURSAL BARRANQUILLA,DAB02982025,2025.0,37,3.0,18.685080,10.0,48785.61,0.210170,3.172414


In [19]:
filtro_bq1.query("product_ref == 'DLS10286189'")

,store_name,product_ref,año,semana_num,num_facturas,EMA,demanda_ajustada,producto_costo_unitario,desviacion_ema%,coef_ventas


In [20]:
filtro_bq1["product_ref"].nunique()

387

In [21]:
import ast
ema_real = pd.read_csv("/home/donsson/proyectos/API/real_ema.csv")

# crear columna "sucursal" a partir del nombre en almacen_id
ema_real["sucursal"] = ema_real["almacen_id"].apply(lambda x: x[1] if isinstance(x, list) else None)

# si almacen_id viene como string "[39, 'BARRANQUILLA']"
ema_real["almacen_id"] = ema_real["almacen_id"].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)

# ahora sí sacar la sucursal
ema_real["sucursal"] = ema_real["almacen_id"].apply(
lambda x: x[1] if isinstance(x, list) and len(x) > 1 else None
)

ema_real = ema_real[["sucursal","product_name","ano","semana","ema","producto_costo_unitario"]]

In [22]:
bq_real = ema_real[ema_real["sucursal"]=="BARRANQUILLA"]
bq_real = bq_real[bq_real["producto_costo_unitario"]>1]
bq_real = bq_real[bq_real["ema"]> 1]
bq_real.head(30)

,sucursal,product_name,ano,semana,ema,producto_costo_unitario
0,BARRANQUILLA,DAB02570025,2025,37,38.481576,13645.86
1,BARRANQUILLA,BLS00037125,2025,37,33.846635,32797.97
2,BARRANQUILLA,DAB02772025,2025,37,30.770240,10552.22
3,BARRANQUILLA,DAB14570025,2025,37,28.342213,9774.98
4,BARRANQUILLA,BCS00035125,2025,37,23.357090,20638.72
5,BARRANQUILLA,DAB02666025,2025,37,22.925359,11371.65
6,BARRANQUILLA,BCS10035125,2025,37,22.016999,20956.60
7,BARRANQUILLA,BCS00025125,2025,37,19.300411,31057.73
8,BARRANQUILLA,DAB02982025,2025,37,18.787264,48785.61
9,BARRANQUILLA,BCS00249125,2025,37,18.653180,27520.25


In [23]:
bq_real["product_name"].nunique()

381

# PROMEDIO DE VENTAS MENSUALES Y VP BUSCADOR

In [30]:
ventas_agrupadas_mes = (
    df_merged
    .groupby(["año","mes","product_ref"])[["sales", "lost_sales"]]
    .sum()
    .reset_index()
)

# Lista de referencias a filtrar
refs = [
    "DCE00833189"

]

# Filtrar por año y referencias
ventas_filtradas = ventas_agrupadas_mes[
    (ventas_agrupadas_mes["año"] == 2025)
    & (ventas_agrupadas_mes["product_ref"].str.contains("|".join(refs)))
]



In [31]:
ventas_filtradas.head(12)

,año,mes,product_ref,sales,lost_sales
8281,2025,1,DCE00833189,180.0,12.0
10863,2025,2,DCE00833189,273.0,6.0
13486,2025,3,DCE00833189,187.0,201.0
15979,2025,4,DCE00833189,212.0,0.0
18364,2025,5,DCE00833189,210.0,0.0
20759,2025,6,DCE00833189,191.0,0.0
23069,2025,7,DCE00833189,175.0,1.0
25467,2025,8,DCE00833189,132.0,168.0
27528,2025,9,DCE00833189,175.0,55.0


In [26]:
#ventas_filtradas.to_excel("promxpec.xlsx")